In [197]:
import pandas as pd
import numpy as np

In [198]:
papas_g1 = pd.read_csv('../Data/papas_g1_diaria.csv')
papas_g2 = pd.read_csv('../Data/papas_g2_diaria.csv')

papas_procedencia_g1 = pd.read_csv('../Data/papas_procedencia_g1_diaria.csv')
papas_procedencia_g2 = pd.read_csv('../Data/papas_procedencia_g2_diaria.csv')

petroleo = pd.read_csv('../Data/precio_petroleo_diario.csv')

temperatura = pd.read_csv('../Data/temperatura_automatico.csv')

In [199]:
dataProcedenciaJunin_g1 = papas_procedencia_g1[papas_procedencia_g1["procedencia"] == "JUNIN"][["cantidad (TM)", "date"]]
dataProcedenciaJunin_g1["date"] = pd.to_datetime(dataProcedenciaJunin_g1["date"], format="%Y-%m-%d")
dataProcedenciaJunin_g1 = dataProcedenciaJunin_g1.groupby('date').sum().reset_index()

dataProcedenciaJunin_g2 = papas_procedencia_g2[papas_procedencia_g2["procedencia"] == "JUNIN"][["cantidad (TM)", "date"]]
dataProcedenciaJunin_g2["date"] = pd.to_datetime(dataProcedenciaJunin_g2["date"], format="%Y-%m-%d")
dataProcedenciaJunin_g2 = dataProcedenciaJunin_g2.groupby('date').sum().reset_index()

papas_g1["date"] = pd.to_datetime(papas_g1["date"], format="%Y-%m-%d")
papas_g2["date"] = pd.to_datetime(papas_g2["date"], format="%Y-%m-%d")


In [200]:
papas_g1 = papas_g1[['producto','precio_mean','date']]
papas_g2 = papas_g2[['producto','precio_mean','date']]

In [201]:
month_map = {
    "Ene": "01",
    "Feb": "02",
    "Mar": "03",
    "Abr": "04",
    "May": "05",
    "Jun": "06",
    "Jul": "07",
    "Ago": "08",
    "Set": "09",
    "Oct": "10",
    "Nov": "11",
    "Dic": "12",
}


def parse_date(row):
    return (
        row["fecha"][:2] + "-" + month_map[row["fecha"][2:5]] + "-" + row["fecha"][5:]
    )


petroleo["date"] = petroleo.apply(lambda row: parse_date(row), axis=1)
petroleo["date"] = pd.to_datetime(petroleo["date"], format="%d-%m-%y")
petroleo = petroleo[["date", "precio"]]
petroleo = petroleo[petroleo["date"] >= "2015-01-01"]
petroleo["precio"] = petroleo.apply(
    lambda row: 0 if row["precio"] == "n.d." else row["precio"], axis=1
)
petroleo["precio"] = petroleo["precio"].astype(float)

In [202]:
dates = pd.DataFrame(papas_g1['date'].unique(), columns=['date']).set_index('date').resample('D').asfreq().index
productos = papas_g1['producto'].unique()
idx = pd.MultiIndex.from_product((dates, productos), names=['dates', 'producto'])
papas_g1_completo = papas_g1.set_index(['date', 'producto']).reindex(idx, fill_value=0).reset_index()
papas_g1_completo['precio_mean'] = papas_g1_completo['precio_mean'].replace(to_replace=0, method='ffill')

dates = pd.DataFrame(papas_g2['date'].unique(), columns=['date']).set_index('date').resample('D').asfreq().index
productos = papas_g2['producto'].unique()
idx = pd.MultiIndex.from_product((dates, productos), names=['dates', 'producto'])
papas_g2_completo = papas_g2.set_index(['date', 'producto']).reindex(idx, fill_value=0).reset_index()
papas_g2_completo['precio_mean'] = papas_g2_completo['precio_mean'].replace(to_replace=0, method='ffill')

In [203]:
papas_g1_completo.to_csv('../Data/Processed/papas_g1.csv', index=False)
papas_g2_completo.to_csv('../Data/Processed/papas_g2.csv', index=False)

dataProcedenciaJunin_g1.to_csv('../Data/Processed/procedencia_g1.csv', index=False)
dataProcedenciaJunin_g2.to_csv('../Data/Processed/procedencia_g2.csv', index=False)

# Solo precio

In [204]:
papas_g1_completo_pre = papas_g1_completo[(
    papas_g1_completo['dates'] >= '2013-12-01') & (papas_g1_completo['dates'] < '2020-01-01')]

papas_g1_completo_post = papas_g1_completo[papas_g1_completo['dates'] >= '2020-01-01']

papas_g1_completo_pre.to_csv('../Data/solo_precio/papas_g1_pre.csv', index=False)
papas_g1_completo_post.to_csv('../Data/solo_precio/papas_g1_post.csv', index=False)

papas_g2_completo_pre = papas_g2_completo[(
    papas_g2_completo['dates'] >= '2013-12-01') & (papas_g2_completo['dates'] < '2020-01-01')]

papas_g2_completo_post = papas_g2_completo[papas_g2_completo['dates'] >= '2020-01-01']

papas_g2_completo_pre.to_csv('../Data/solo_precio/papas_g2_pre.csv', index=False)
papas_g2_completo_post.to_csv('../Data/solo_precio/papas_g2_post.csv', index=False)

# Precio + Petroleo + Ingreso total

In [205]:
# Precio
papas_g1_completo_pre = papas_g1_completo[(
    papas_g1_completo['dates'] >= '2015-01-01') & (papas_g1_completo['dates'] < '2020-01-01')]

papas_g1_completo_post = papas_g1_completo[papas_g1_completo['dates'] >= '2020-01-01']

papas_g1_completo_pre.to_csv('../Data/precio_petroleo_cantidad/papas_g1_pre.csv', index=False)
papas_g1_completo_post.to_csv('../Data/precio_petroleo_cantidad/papas_g1_post.csv', index=False)

papas_g2_completo_pre = papas_g2_completo[(
    papas_g2_completo['dates'] >= '2015-01-01') & (papas_g2_completo['dates'] < '2020-01-01')]

papas_g2_completo_post = papas_g2_completo[papas_g2_completo['dates'] >= '2020-01-01']

papas_g2_completo_pre.to_csv('../Data/precio_petroleo_cantidad/papas_g2_pre.csv', index=False)
papas_g2_completo_post.to_csv('../Data/precio_petroleo_cantidad/papas_g2_post.csv', index=False)

# Petroleo
index_pre = pd.DatetimeIndex(pd.date_range('2015-01-01','2019-12-31'))
index_post = pd.DatetimeIndex(pd.date_range('2020-01-01','2022-08-31'))

petroleo_pre = petroleo[(
    petroleo['date'] >= '2015-01-01') & (petroleo['date'] < '2020-01-01')]
petroleo_pre = petroleo_pre.set_index('date').reindex(index_pre, fill_value=0).reset_index()
petroleo_pre['precio'] = petroleo_pre['precio'].replace(to_replace=0, method='ffill')

petroleo_post = petroleo[petroleo['date'] >= '2020-01-01']
petroleo_post = petroleo_post.set_index('date').reindex(index_post, fill_value=0).reset_index()
petroleo_post['precio'] = petroleo_post['precio'].replace(to_replace=0, method='ffill')

petroleo_pre.to_csv('../Data/precio_petroleo_cantidad/petroleo_pre.csv', index=False)
petroleo_post.to_csv('../Data/precio_petroleo_cantidad/petroleo_post.csv', index=False)

# Cantidad
papas_procedencia_g1_total = papas_procedencia_g1[['date','cantidad (TM)']].groupby('date').sum().reset_index()

papas_procedencia_g1_total_pre = papas_procedencia_g1_total[(
    papas_procedencia_g1_total['date'] >= '2015-01-01') & (papas_procedencia_g1_total['date'] < '2020-01-01')]

papas_procedencia_g1_total_post = papas_procedencia_g1_total[papas_procedencia_g1_total['date'] >= '2020-01-01']

papas_procedencia_g1_total_pre.to_csv('../Data/precio_petroleo_cantidad/papas_procedencia_g1_total_pre.csv', index=False)
papas_procedencia_g1_total_post.to_csv('../Data/precio_petroleo_cantidad/papas_procedencia_g1_total_post.csv', index=False)

papas_procedencia_g2_total = papas_procedencia_g2[['date','cantidad (TM)']].groupby('date').sum().reset_index()

papas_procedencia_g2_total_pre = papas_procedencia_g2_total[(
    papas_procedencia_g2_total['date'] >= '2015-01-01') & (papas_procedencia_g2_total['date'] < '2020-01-01')]

papas_procedencia_g2_total_post = papas_procedencia_g2_total[papas_procedencia_g2_total['date'] >= '2020-01-01']

papas_procedencia_g2_total_pre.to_csv('../Data/precio_petroleo_cantidad/papas_procedencia_g2_total_pre.csv', index=False)
papas_procedencia_g2_total_post.to_csv('../Data/precio_petroleo_cantidad/papas_procedencia_g2_total_post.csv', index=False)


# Precio + Petroleo + Ingreso Junin + Temperatura Junin

In [206]:
# Precio
papas_g1_completo_pre = papas_g1_completo[(
    papas_g1_completo['dates'] >= '2018-01-01') & (papas_g1_completo['dates'] < '2020-01-01')]

papas_g1_completo_post = papas_g1_completo[papas_g1_completo['dates'] >= '2020-01-01']

papas_g1_completo_pre.to_csv('../Data/precio_petroleo_junin/papas_g1_pre.csv', index=False)
papas_g1_completo_post.to_csv('../Data/precio_petroleo_junin/papas_g1_post.csv', index=False)

papas_g2_completo_pre = papas_g2_completo[(
    papas_g2_completo['dates'] >= '2018-01-01') & (papas_g2_completo['dates'] < '2020-01-01')]

papas_g2_completo_post = papas_g2_completo[papas_g2_completo['dates'] >= '2020-01-01']

papas_g2_completo_pre.to_csv('../Data/precio_petroleo_junin/papas_g2_pre.csv', index=False)
papas_g2_completo_post.to_csv('../Data/precio_petroleo_junin/papas_g2_post.csv', index=False)

# Petroleo
index_pre = pd.DatetimeIndex(pd.date_range('2018-01-01','2019-12-31'))
index_post = pd.DatetimeIndex(pd.date_range('2020-01-01','2022-08-31'))

petroleo_pre = petroleo[(
    petroleo['date'] >= '2018-01-01') & (petroleo['date'] < '2020-01-01')]
petroleo_pre = petroleo_pre.set_index('date').reindex(index_pre, fill_value=0).reset_index()
petroleo_pre['precio'] = petroleo_pre['precio'].replace(to_replace=0, method='ffill')

petroleo_post = petroleo[petroleo['date'] >= '2020-01-01']
petroleo_post = petroleo_post.set_index('date').reindex(index_post, fill_value=0).reset_index()
petroleo_post['precio'] = petroleo_post['precio'].replace(to_replace=0, method='ffill')

petroleo_pre.to_csv('../Data/precio_petroleo_junin/petroleo_pre.csv', index=False)
petroleo_post.to_csv('../Data/precio_petroleo_junin/petroleo_post.csv', index=False)

# Cantidad Junin

dataProcedenciaJunin_g1_pre = dataProcedenciaJunin_g1[(
    dataProcedenciaJunin_g1['date'] >= '2018-01-01') & (dataProcedenciaJunin_g1['date'] < '2020-01-01')]

dataProcedenciaJunin_g1_post = dataProcedenciaJunin_g1[dataProcedenciaJunin_g1['date'] >= '2020-01-01']

dataProcedenciaJunin_g1_pre.to_csv('../Data/precio_petroleo_junin/cantidad_junin_g1_pre.csv', index=False)
dataProcedenciaJunin_g1_post.to_csv('../Data/precio_petroleo_junin/cantidad_junin_g1_post.csv', index=False)

dataProcedenciaJunin_g2_pre = dataProcedenciaJunin_g2[(
    dataProcedenciaJunin_g2['date'] >= '2018-01-01') & (dataProcedenciaJunin_g2['date'] < '2020-01-01')]

dataProcedenciaJunin_g2_post = dataProcedenciaJunin_g2[dataProcedenciaJunin_g2['date'] >= '2020-01-01']

dataProcedenciaJunin_g2_pre.to_csv('../Data/precio_petroleo_junin/cantidad_junin_g2_pre.csv', index=False)
dataProcedenciaJunin_g2_post.to_csv('../Data/precio_petroleo_junin/cantidad_junin_g2_post.csv', index=False)

# Temperatura Junin
temperatura_junin = temperatura[temperatura['DEPARTAMENTO'] == 'JUNIN'][['FECHA','TEMPERATURA']]
temperatura_junin['FECHA'] = pd.to_datetime(temperatura_junin['FECHA'], format='%Y-%m-%d')
temperatura_junin = temperatura_junin[temperatura_junin['TEMPERATURA'] != 'S/D']
temperatura_junin['TEMPERATURA'] = temperatura_junin['TEMPERATURA'].astype(float)
temperatura_junin = temperatura_junin.groupby('FECHA').mean().reset_index()

temperatura_junin_pre = temperatura_junin[(
    temperatura_junin['FECHA'] >= '2018-01-01') & (temperatura_junin['FECHA'] < '2020-01-01')]

temperatura_junin_post = temperatura_junin[temperatura_junin['FECHA'] >= '2020-01-01']

temperatura_junin_pre.to_csv('../Data/precio_petroleo_junin/temperatura_junin_pre.csv', index=False)
temperatura_junin_post.to_csv('../Data/precio_petroleo_junin/temperatura_junin_post.csv', index=False)